# Homework 1 - Wikipedia Web Traffic Time Series

У вас есть данные по посещению 1000 страниц  Википедии из разных стран и разных девайсов ( \*  * данные взяты из [Kaggle соревнования](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train* и *wikipedia_test* - содержат данные о трафике. Это файлы csv, где каждая строка соответствует определенной статье, и каждый столбец соответствует конкретной дате. В некоторых записях отсутствуют данные. Названия страниц содержат проект Википедии (например, en.wikipedia.org), тип доступа (например, desktop) и тип агента (например, spider). Другими словами, каждое имя статьи имеет следующий формат: «name_project_access_agent» (например, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам нужно ответить на [вопросы](https://docs.google.com/forms/d/e/1FAIpQLSfDjWeeZJw5EvmKn1x_6b9xicjn7ed3MF0rbNm4Cmwr7psSkQ/viewform?usp=sf_link) и попробовать сделать самую простую модель которая сможет предсказывать будущие посещения. 

Вот примеры временных рядов посещаемости страниц Википедии (*синие* - обучающая выборка, *зеленые* - предсказания модели победителя соревнования на Kaggle, *оранжевые* - реальные значения):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [1]:
import re
import pandas as pd
import numpy as np

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("wikipedia_train.csv")
test = pd.read_csv("wikipedia_test.csv")

In [153]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-08-22,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31
0,15._November_de.wikipedia.org_desktop_all-agents,32.0,26.0,22.0,22.0,29.0,49.0,20.0,27.0,19.0,...,29.0,23.0,31.0,25.0,27.0,23.0,17.0,26.0,23.0,37.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2.0,3.0,5.0,3.0,5.0,3.0,7.0,8.0,7.0,...,5.0,5.0,6.0,5.0,4.0,11.0,2.0,0.0,7.0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,1.0,3.0,2.0,2.0,1.0,10.0,2.0,1.0,4.0,...,4.0,3.0,2.0,3.0,2.0,4.0,2.0,0.0,5.0,4.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,3.0,3.0,3.0,8.0,12.0,12.0,8.0,12.0,23.0,...,10.0,14.0,26.0,5.0,29.0,23.0,17.0,16.0,12.0,14.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,24.0,40.0,23.0,49.0,88.0,25.0,31.0,76.0,51.0,...,134.0,162.0,208.0,179.0,108.0,99.0,49.0,80.0,113.0,173.0


In [49]:
train.shape, test.shape

((939, 429), (939, 63))

In [90]:
test.head()

,Page,2016-09-10,2016-09-11,2016-09-12,2016-09-13,2016-09-14,2016-09-15,2016-09-16,2016-09-17,2016-09-18,...,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08,2016-11-09,2016-11-10
0,15._November_de.wikipedia.org_desktop_all-agents,43.0,29.0,19.0,33.0,30.0,28.0,27.0,15.0,22.0,...,66.0,78.0,119.0,78.0,63.0,89.0,141.0,146.0,120.0,147.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,6.0,31.0,23.0,12.0,1.0,24.0,9.0,30.0,4.0,...,2.0,0.0,3.0,2.0,6.0,1.0,2.0,1.0,3.0,2.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,4.0,2.0,0.0,5.0,3.0,2.0,4.0,3.0,0.0,...,16.0,17.0,11.0,18.0,20.0,25.0,13.0,19.0,22.0,33.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,15.0,15.0,8.0,11.0,15.0,14.0,13.0,18.0,18.0,...,14.0,13.0,14.0,20.0,16.0,11.0,26.0,16.0,11.0,13.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,101.0,35.0,80.0,191.0,199.0,301.0,174.0,155.0,136.0,...,94.0,104.0,85.0,108.0,110.0,109.0,221.0,75.0,63.0,101.0


## Data Analysis

In [11]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

In [12]:
train['language'] = train['Page'].apply(lambda x: get_language(x))
trainru = train[train.language == 'ru']
trainru.drop(['language', 'Page'], axis = 1, inplace=True)
trainru.fillna(0, inplace=True)
trainru['meanvisits'] = trainru.apply(np.mean, axis = 1)

In [13]:
trainru.sort_values(by = 'meanvisits',ascending=False).head(1)

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31,meanvisits
180,1665.0,1419.0,1440.0,1062.0,1169.0,1496.0,1509.0,1602.0,1383.0,1341.0,...,2151.0,1786.0,1817.0,1927.0,1401.0,1318.0,1708.0,1748.0,2235.0,2171.240654


In [14]:
train.loc[180,'Page']

'Facebook_ru.wikipedia.org_desktop_all-agents'

## Forecasting

Нужно преобразовать `train` данные в следующий формат:

In [13]:
train1 = pd.melt(train, value_vars=train.columns[1:], id_vars='Page')
train1.head()

,Page,variable,value
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24.0


Таким образом у вас каждая сточка содержит набор фич (`Page`, `date`) и целевую переменную (`Visits`). Преобразовать данные в такой формат поможет функция `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

Оценивать качество предсказаний мы будем с помощью [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [5]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

Что б оценить прогноз нужно представить `test` данные в следующем формате:

И затем вызвать функцию для подсчета метрики:

In [ ]:
pandas_smape(prediction)

### Last day baseline

Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test)

In [7]:
train.fillna(0, inplace=True)

In [8]:
#test_pred = pd.DataFrame({i:train.loc[:,'2016-08-31'].values for i in test.columns[1:]})

test_pred = pd.DataFrame({i:np.median(train.iloc[:,-30:], axis = 1) for i in test.columns[1:]})

test_pred['Page'] = test.Page
test_predm = pd.melt(test_pred, value_vars=test_pred.columns[:-1], id_vars='Page')

testm = pd.melt(test, value_vars=test.columns[1:], id_vars='Page')
testm['pred_Visits'] = test_predm.value
testm.rename(columns={'variable':'date', 'value':'Visits'}, inplace = True)

pandas_smape(testm)

51.89889768202033

In [9]:
for j in range(1,61):
    test_pred = pd.DataFrame({i:np.median(train.iloc[:,((-1)*j):], axis = 1) for i in test.columns[1:]})

    test_pred['Page'] = test.Page
    test_predm = pd.melt(test_pred, value_vars=test_pred.columns[:-1], id_vars='Page')

    testm = pd.melt(test, value_vars=test.columns[1:], id_vars='Page')
    testm['pred_Visits'] = test_predm.value
    testm.rename(columns={'variable':'date', 'value':'Visits'}, inplace = True)

    print(j, pandas_smape(testm))

1 54.16127748085736
2 52.605789626254406
3 51.535987145908415
4 51.32789623733235
5 50.70362817774352
6 50.67609105665244
7 50.312225925288416
8 50.09678254308233
9 49.842494249794555
10 49.85240803464807
11 49.79945300195831
12 49.98347020174622
13 50.0445948379971
14 50.151430485294625
15 50.13711273380181
16 50.48926186140007
17 50.565702736225255
18 51.282409460155485
19 51.45280170179484
20 51.63698705380347
21 51.59009210852701
22 51.61371751966444
23 51.646843106420974
24 51.78785607670561
25 51.77262205224886
26 51.86342601345956
27 51.83060229153667
28 51.94107338294196
29 51.81376360232597
30 51.89889768202033
31 51.74728528812765
32 51.86256847236707
33 51.82441967688524
34 51.838560544454566
35 51.79362464780881
36 51.79880674302419
37 51.66497364671561
38 51.69013401058317
39 51.59095540184257
40 51.62460113913027
41 51.54604247657092
42 51.60553687490505
43 51.48320970697684
44 51.685379444046944
45 51.57382074824785
46 51.680819114312094
47 51.62224081277162
48 51.751670

### Median baseline

Нужно сделать прогноз на основе медианы за последние **30** дней из `train`. 

А затем улучшить предсказания используя информацию выходной это или нет (воспользуйтесь функцией [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) и разные окна для подсчета медианы (7 дней, 60 дней и тд)

Вам поможет функция `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

In [186]:
from datetime import datetime, date, time

train = pd.read_csv("wikipedia_train.csv")
train.fillna(0, inplace=True)


train_weekday_columns = np.array([datetime.strptime(i, "%Y-%m-%d").isoweekday() for i in train.columns[1:]])
train_workday = train.iloc[:, (train_weekday_columns != 6)&(train_weekday_columns != 7)]
train_weekend = train.iloc[:, (train_weekday_columns == 6)|(train_weekday_columns == 7)]

test_weekday_columns = np.array([datetime.strptime(i, "%Y-%m-%d").isoweekday() for i in test.columns[1:]])
test_workday = test.iloc[:, (test_weekday_columns != 6)&(test_weekday_columns != 7)]
test_weekend = test.iloc[:, (test_weekday_columns == 6)|(test_weekday_columns == 7)]

smape_results = pd.DataFrame({i:np.ones(20) for i in range(6)})

for work in range(smape_results.shape[0]):
    for end in range(smape_results.shape[1]):

        test_pred_workday = pd.DataFrame({i:np.median(train_workday.iloc[:,1:].iloc[:,(-1)*work:], axis = 1) for i in test_workday.columns})
        test_pred_weekend = pd.DataFrame({i:np.median(train_weekend.iloc[:,1:].iloc[:,(-1)*end:], axis = 1) for i in test_weekend.columns})

        test_pred = pd.concat([test_pred_workday, test_pred_weekend], axis=1)


        test_predm = pd.melt(test_pred, value_vars=test_pred.columns[:-1], id_vars='Page')
        test_predm.sort_values(by = ['variable'], inplace = True)
        testm = pd.melt(test, value_vars=test.columns[1:], id_vars='Page')
        testm['pred_Visits'] = test_predm.value
        testm.rename(columns={'variable':'date', 'value':'Visits'}, inplace = True)

        smape_results.iloc[work, end] = pandas_smape(testm)
        
smape_results

,0,1,2,3,4,5
0,54.002589,56.502986,55.764152,55.386954,55.392198,55.422883
1,55.775309,58.275705,57.536871,57.159673,57.164917,57.195603
2,55.077931,57.578328,56.839494,56.462296,56.467540,56.498226
3,53.971921,56.472318,55.733484,55.356286,55.361530,55.392216
4,53.699531,56.199927,55.461093,55.083895,55.089139,55.119825
5,53.055447,55.555844,54.817009,54.439812,54.445056,54.475741
6,53.081970,55.582366,54.843532,54.466334,54.471578,54.502264
7,52.786949,55.287345,54.548511,54.171313,54.176557,54.207243
8,52.713621,55.214018,54.475184,54.097986,54.103230,54.133915
9,52.717657,55.218053,54.479219,54.102021,54.107265,54.137951
